# Analiza danych: regresja X->Y.

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np

from sklearn import metrics

import os

In [ ]:
folder_data = "data" # tu wpisz adres folderu, gdzie zapisałeś dane

from collections import defaultdict
from itertools import combinations
import scipy.stats as stats
import matplotlib.pyplot as plt

import seaborn as sns

# Dane treningowe

In [ ]:
X = pd.read_hdf(os.path.join(folder_data, "X.h5"), "data")

In [ ]:
Y = pd.read_hdf(os.path.join(folder_data, "Y.h5"), "data")

In [ ]:
Y.hist(bins=100)

In [ ]:
X.shape, Y.shape

# Loss function (funkcja straty / kryterium dopasowania) to R^2 -> max R^2

In [ ]:
def metric(Y, Y_pred):
    return metrics.r2_score(Y, Y_pred)

In [ ]:
# Zmienne pomocnicze

# Lista nazw predykatorów.
PREDICTORS_NAMES = frozenset(X.columns.values)

STATISTIC_TEST_NAMES = frozenset([
    'Pearson',
    'Spearman '    
])

STATISTIC_TESTS = {
    'Pearson': stats.normaltest,
    'Shapiro': stats.shapiro
}

MIN_SIZE_PREDICTORS = 2
MAX_SIZE_PREDICTORS = 15

# Kombinacje predykatorów o rozmiarze 2.
PREDICTOR_COMBINATIONS = list(combinations(PREDICTORS_NAMES, 2))

STATISTIC_THRESHOLD = 2000

ALPHA = 0.05

## Testy statystyczne sprawdzające czy dany predyktor ma rozkład Gaussa.

Testy ,,normalności" są używane do określenia czy zbiór danych jest dobrze modelowany przez rozkład normalny i obliczenia prawdopodobieństwa normalnego rozkładu zmiennej losowej leżącej u podstaw zbioru danych.

### Interpretacja testu czy dana zmienna ma rozkład normalny (Gaussa)
W każdym teście obliczamy wartość statystyki i p-wartość (prawdopodobieństwo).
 - Statystyka (`statistic`): Wielkość obliczona przez test, którą można zinterpretować w kontekście testu porównując ją z wartościami krytycznymi z rozkładu statystyki testowej.
 
- p-wartość (`p_value`): interpreter testu, który pozwala weryfikować hipotezę statystyczną, w tym przypadku czy zmienna ma rozkład normalny.

Testy zakładają, że próbka została pobrana z rozkładu Gaussa. Technicznie nazywa się to hipotezą zerową lub H0. Wybiera się poziom progowy zwany alfą (`ALPHA`), zwykle jest to 5% (lub 0.05), który jest używany do interpretacji wartości p.

W bibliotece SciPy, w implementacji tych testów - p-wartość należy rozpratrywać następująco

- p <= alpha: odrzucenie hipotezy H0, rozkład zmiennej nie jest normalny (Gaussa)
- p > alpha: nie ma podstaw do odrzucenia hipotezy H0, zmienna ma rozkład normalny.

In [ ]:
# Dodałem próg dla wartości statystki, bez tego otrzymywałem jeden rezultat.
# Wydaje mi się to podejrzane, dlatego też to zrobiłem. Kilka komórek poniżej
# wydrukowuje wykresy dla tych zmiennych (predykatorów).
# Dobrym pytanie jest dla taka wartość progu dla statystyki. Analizując dla każdego
# predykatu wartośc statystyki stwierdziłem, że taki próg będzie ok - bo wartość statystyk
# była bardzo ,,szeroka".

# Funkcje analizujące wyniki normality test.
def analyse_pearson_normal_test(entry) -> bool:
    statistic = entry['statistic']
    p_value = entry['p_value']
    return True if (p_value > ALPHA or statistic < STATISTIC_THRESHOLD) else False

def analyse_shapiro_normal_test(entry) -> bool:
    statistic = entry['statistic']
    p_value = entry['p_value']
    return True if p_value > ALPHA else False

# Klasyfikacja korelacji liniowej Pearsona
def classification_correlation(classified_correlations: dict, correlations: dict) -> dict:
    '''
    classified_correlations -> słownik
    correlations -> słownik z korelacjami dwóch zmiennych, gdzie ,,kluczem'' (key) 
    w tej hash mapie jest string jako konkatenacja nazwy dwóch zmiennych,
    wartością (value) jest wartości współczynnika korelacji liniowej.
    '''
    
    for name, value in correlations.items():
        if 0 < value < 0.1:
            classified_correlations['VERY_WEAK'].append(name)
        elif 0.1 <= value < 0.3:
            classified_correlations['WEAK'].append(name)
        elif 0.3 <= value < 0.5:
            classified_correlations['AVERAGE'].append(name)
        elif 0.5 <= value < 0.7:
            classified_correlations['HIGH'].append(name)
        elif 0.7 <= value < 0.9:
            classified_correlations['VERY_HIGH'].append(name)
        elif 0.1 <= value < 1:
            classified_correlations['ALMOST_FULL'].append(name)
        elif value == 1:
            classified_correlations['PERFECT'].append(name)
    
    return classified_correlations

In [ ]:
statistic_test = {
    'Pearson': [],
    'Shapiro': []
}

# Wykonanie testów czt dany predykator ma rozkład normalny (Gaussa).
for predictor_name in PREDICTORS_NAMES:
    for statistic_name, test in STATISTIC_TESTS.items():
        statistic, p_value = test(X[predictor_name])
        item = { 'name': predictor_name, 'statistic': statistic, 'p_value': p_value }
        statistic_test[statistic_name].append(item)

In [ ]:
# Filtracja testów
for statistic_name, tests in statistic_test.items():
    statistic_test[statistic_name] = sorted(tests, key=lambda entry: entry['statistic'])
    if statistic_name == 'Pearson':
        statistic_test[statistic_name] = list(filter(analyse_pearson_normal_test, tests))
    elif statistic_name == 'Shapiro':
        statistic_test[statistic_name] = list(filter(analyse_shapiro_normal_test, tests))
        
pearson_tests = statistic_test['Pearson']
shapiro_tests = statistic_test['Shapiro']

print(f'Pearson normality tests count -> {len(pearson_tests)}')
print(f'Shapiro normality tests count -> {len(shapiro_tests)}')

In [ ]:
# Wyodrębniłem wybrane predykatory, które są zbliżone do rozkłady normalnego.
selected_predictors = set()
for entry in pearson_tests:
    selected_predictors.add(entry['name'])
    
for entry in shapiro_tests:
    selected_predictors.add(entry['name'])
    
print(f"Selected predictors based on normality test -> \n {*selected_predictors,}")

In [ ]:
# Wizualizuje zmienne, dla których test statystyczny określił, że mają
# rozkład normalny. (Upewniam się czy nie popełniłem wcześnie błędu w podejściu)
for selected_predictor in selected_predictors:
    plt.figure()
    X[selected_predictor].hist(bins=100)

In [ ]:
# Obliczam średnią dla każdego predyktora
mean_predictors = defaultdict()
for predictor_name in PREDICTORS_NAMES:
    mean_predictors[f'mean_{predictor_name}'] = X[predictor_name].mean()
    
# Obliczam korelację liniową Pearsona między dwoma zmiennymi.
predictors_correlation = defaultdict()
for p1, p2 in PREDICTOR_COMBINATIONS:
    predictors_correlation[f'r_{p1}_{p2}'] = abs(stats.pearsonr(X[p1], X[p2])[0])
#     predictors_correlation[f'r_{p1}_{p2}'] = np.corrcoef(X[p1], X[p2])[0, 1]

#### Klasyfikacja współczynnika korelacji liniowej Pearsona

Niech *r* oznacza wartość korelacji liniowej Pearsona.

* r = 0 -> brak zależności
*  0  < |r| < 0.1 -> korelacja nikła
* 0.1 <= |r| < 0.3 -> korelacja słaba
* 0.3 <= |r| < 0.5 -> korelacja przeciętna
* 0.5 <= |r| < 0.7 -> korelacja wysoka
* 0.7 <= |r| < 0.9 -> korelacja bardzo wysoka
* 0.9 <= |r| <  1  -> korelacja prawie pełna
* |r| = 1 -> ,,doskonała" zależność

Oczywiście, że wartości korelacji, które wyznaczają brak zależności liniowej nie wykluczają, że zmienne (predykatory) mogą być zależne, ale zależność ta jest krzywoliniowa. Z drugiej strony wysoka wartość korelacji nie oznacza jednoznacznie, że istnieje duża zależność liniowa między zmiennymi (predykatorami). Może to być spowodowane istnieniem innej zmiennej lub zmiennychm, które między tymi predykatorami są silnie skolerowane.

Myślę, że jest to punkt odniesienia to zbudowana modelu, wzoru.

In [ ]:
# Oceniam korelację między dwoma zmiennymi
classified_correlation_predictors = {
    'VERY_WEAK': [],
    'WEAK': [],
    'AVERAGE': [],
    'HIGH': [],
    'VERY_HIGH': [],
    'ALMOST_FULL': [],
    'PERFECT': []
}

In [ ]:
# Oceniam korelację między dwoma zmiennymi
classified_correlation_predictors = classification_correlation(classified_correlation_predictors, predictors_correlation)

for classificator, predictor_names in classified_correlation_predictors.items():
    print(f'{classificator} -> {len(predictor_names)}')

### Heatmapa dotycząca korelacji zmiennych

In [ ]:
corr = X.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

### Korelacja między predykatorem a wartości Y

In [ ]:
predictor_target_corrlation = defaultdict()
for predictor_name in PREDICTORS_NAMES:
    predictor_target_corrlation[f'r_{predictor_name}_y'] = np.corrcoef(X[predictor_name], Y)[0, 1]


In [ ]:
# Oceniam korelację między dwoma predykatorem a wartością Y
classified_correlation_target_predictors = {
    'VERY_WEAK': [],
    'WEAK': [],
    'AVERAGE': [],
    'HIGH': [],
    'VERY_HIGH': [],
    'ALMOST_FULL': [],
    'PERFECT': []
}

In [ ]:
# Oceniam korelację między dwoma zmiennymi
classified_correlation_target_predictors = classification_correlation(classified_correlation_target_predictors, predictor_target_corrlation)

for classificator, predictor_names in classified_correlation_target_predictors.items():
    print(f'{classificator} -> {len(predictor_names)}')

### Wyodrębnienie predykatorów z już klasyfikowanymi wartościami korelacji

Interesują mnie przede następujące zależności:
   * PERFECT
   * ALMOST_FULL
   * VERY_HIGH
   * HIGH

In [ ]:
# Obliczam ilość wystąpień predykatów w powyżej wymienionych.
first_statitistic_correlation_predictors = defaultdict(int)
second_statitistic_correlation_predictors = defaultdict(int)
selected_keys = frozenset(['PERFECT', 'ALMOST_FULL', 'VERY_HIGH', 'HIGH'])
for key, items in classified_correlation_predictors.items():
    if key not in selected_keys:
        continue
    
    for item in items:
        first_pred, second_pred = item.split('_')[1:]
        first_statitistic_correlation_predictors[first_pred] += 1
        second_statitistic_correlation_predictors[second_pred] += 1

# Obliczam średnią wystąpień predykatu w danej klasyfikacji.
first_treshold_mean = np.mean(list(first_statitistic_correlation_predictors.values()))
second_treshold_mean = np.mean(list(second_statitistic_correlation_predictors.values()))

# Wartości wystąpień predykatu powyżej średniej dorzucam do już wyselekcjonowanych
# predykatów.
for predictor, count in first_statitistic_correlation_predictors.items():
    if count > first_treshold_mean:
        selected_predictors.update(predictor)

for predictor, count in second_statitistic_correlation_predictors.items():
    if count > second_treshold_mean:
        selected_predictors.update(predictor)

### Wyodrębnienie predykatorów z obliczeń korelacji z wartością Y (target)

Ze wcześniejszych obliczeń wyszło, że są dwie klasyfikacje `VERY_WEAK` i `WEAK`. Co oznacza, że jest poniżej przeciętnej. Wniosek stąd taki, że bierzemy tylko predykatory z ,,szufladki" `WEAK`.

In [ ]:
# ,,Wyciągam" predykaty.
statitistic_correlation_target_predictors = set()
selected_keys = frozenset(['WEAK'])
for key, items in classified_correlation_target_predictors.items():
    if key not in selected_keys:
        continue
    
    for item in items:
        predictor = item.split('_')[1]
        statitistic_correlation_target_predictors.add(predictor)

In [ ]:
# Wyciągam wartość wspólną z dwóch zbiorów predykatów.
statitistic_correlation_target_predictors.intersection_update(selected_predictors)

### Analizując dane, wyciągnąłem predykaty, które wykorzystam do zbudowania modelu.

Oto one ->
`f1`, `f15`, `f30`, `f45`, `f66`, `f134`, `f198`, `f202`,
`f207`, `f208`, `f209`, `f211`, `f212`, `f213`, `f221`, `f259`,
`f260`, `f267`, `f268`, `f275`, `f276`, `f280`, `f284`, `f288`,
`f289`, `f290`, `f291`, `f292`

In [ ]:
SELECTED_PREDICTORS = frozenset(statitistic_correlation_target_predictors)
SELECTED_X_DATA = X[SELECTED_PREDICTORS]

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(SELECTED_X_DATA, Y, test_size=0.3, random_state=1)